In [24]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Collecting xgboost
  Downloading xgboost-1.7.3-py3-none-win_amd64.whl (89.1 MB)
     --------------------------------------- 89.1/89.1 MB 12.8 MB/s eta 0:00:00
Requirement already satisfied: numpy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.21.5)
Requirement already satisfied: scipy in c:\users\mosac\anaconda3\lib\site-packages (from xgboost) (1.9.1)
Installing collected packages: xgboost
Successfully installed xgboost-1.7.3
Note: you may need to restart the kernel to use updated packages.

In [25]:
%pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
%pip install mediapipe 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\Orel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from  xgboost import XGBClassifier
import xgboost
from sklearn.metrics import r2_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [8]:
def calculate_angles(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #middle
    c = np.array(c) #last
    #Calculates the angle formed by the three dots
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)#Convert to degrees
    #An angle cannot be more than 180
    if angle > 180.0:
        angle = 360- angle

    return angle

Direction of trainer in the image 

Calculate the angels

In [4]:
#3 angale
#every anagle creates from:a,b,c
data = {
    'elbow': [],    
    'knee': [],    
    'shoulder': [],    
    'label': [] #The value of the character in terms of the X-axis
}

#run over all the images in images folder
directory = r'C:\Users\Orel\Documents\GitHub\FitnessAI\images'
label = ''
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
         
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
         
            #run over the images inside the folders
            #takes every image and put it inside a frame,f=image
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)#read the image to frame of cv2

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#paint the pose
            image.flags.writeable = False
      
            results = pose.process(image)#the picther with the scale and dots

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#change the color format

            #Extract  the dots from the image,every dots contain x,y,z,visabillity and saved in arr
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass
             
            #drew the dots
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), #different color-blue
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) #different color-pink
                            )
            # Get coordinates 
            R_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            R_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            R_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
            #Visibility Extraction
            R_K_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            R_E_V = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            R_S_V = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            

            L_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            L_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            L_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            #Visibility Extraction
            L_K_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            L_E_V = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            L_S_V = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
                

                # Calculate angles
            if  R_K_V > L_K_V:
                angle_K = calculate_angles(R_hip, R_knee, R_ankle)
            else:
                angle_K = calculate_angles(L_hip, L_knee, L_ankle)

            if R_E_V > L_E_V:
                angle_E = calculate_angles(R_shoulder, R_elbow, R_wrist)
            else:
                angle_E = calculate_angles(L_shoulder, L_elbow, L_wrist)
            
            if  R_S_V >  L_S_V:
                angle_S = calculate_angles(R_elbow, R_shoulder, R_hip)
            else:
                angle_S = calculate_angles(L_elbow, L_shoulder, L_hip)
        

         #binary classification between d,s,b
         #ask in the folder give label by the name written in the image
        if 'deadlift' in f:
            label = 'deadlift'
        elif 'squat' in f:
            label = 'squat'
        elif 'bench' in f:
            label = 'bench'
        else:
            continue
      
        #add those angels values to the data array.
    data["elbow"].append(angle_E)
    data["knee"].append(angle_K)
    data["shoulder"].append(angle_S) 
    data["label"].append(label)       

            


In [5]:
data

{'elbow': [174.32559264386725,
  134.7152564233026,
  151.18138682711594,
  156.83443576303182,
  8.316940874659196,
  106.22452378664772,
  0.4643140035979646,
  174.23232377309012,
  177.41162448958846,
  178.75060814727402,
  7.8561569442724855,
  176.39468789909486,
  21.486908115274574,
  170.83800826112713,
  170.83800826112713,
  96.80943879139312,
  176.58345504840483,
  176.58345504840483,
  50.708656859881536,
  165.0284802019167,
  4.901152388267193,
  164.32469211802004,
  156.83443576303182,
  69.42366356440694,
  170.83800826112713,
  160.79111497079816,
  162.29245957958506,
  179.9698592934558,
  179.35918982607572,
  50.23672129119279,
  66.65622502161602,
  155.03448484802567,
  175.02346227419196,
  179.41896852308736,
  163.70435495770596,
  172.82795922372065,
  19.04178428539857,
  174.83744260670204,
  174.83744260670204,
  166.14136362238955,
  166.4722391372106,
  2.8980926153628785,
  33.32506555716228,
  164.09083309719344,
  169.8651768791044,
  39.250180925

In [6]:
#Make data array into pandas DataFrame array
df = pd.DataFrame(data=data)

In [11]:
#Encoding categorial features
le = LabelEncoder()
le.fit(df.loc[:,'label'])
df.loc[:,'label'] = le.transform(df.loc[:,'label'])
df.tail()


C:\Users\Orel\AppData\Local\Temp\ipykernel_24368\2942440658.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'label'] = le.transform(df.loc[:,'label'])


,elbow,knee,shoulder,label
281,48.113965,162.201790,41.110027,2
282,31.535423,179.439008,43.293464,2
283,72.571456,173.702683,57.322578,2
284,48.115591,149.335647,33.674356,2
285,45.321530,157.426115,31.453297,2


In [12]:
#train
X = df.drop(['label'],axis=1)#3 angels
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_test,y_test = shuffle(X_test,y_test)
X_train,y_train = shuffle(X_train,y_train)


In [17]:
xgb_model = XGBClassifier(learning_rate=1, n_estimators= 100,max_depth = 5)

fit = xgb_model.fit(X_train,y_train)

fit_pred = fit.predict(X_train)

xgb_predictions = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test,xgb_predictions)

#Evaluation - calculating accuracy
xgb_train_acc = accuracy_score(y_train,fit_pred)

#Print accuracy of train and test
print(f"Train Accuracy Is: {round(xgb_train_acc*100,2)}\nTest Accuracy Is: {round(xgb_accuracy*100,2)}")

print(xgb_predictions)
print(y_test.values)


Train Accuracy Is: 100.0
Test Accuracy Is: 91.58
[0 0 0 0 0 0 0 2 0 2 0 0 0 1 0 1 2 0 0 0 1 2 2 1 0 2 2 0 0 0 2 1 2 2 2 1 1
 0 1 0 0 2 0 0 0 0 0 0 2 0 2 1 0 2 2 2 0 2 0 2 1 1 0 0 0 2 2 1 0 0 1 2 1 0
 1 0 2 0 0 1 0 0 0 1 2 1 0 0 1 0 0 1 0 1 2]
[0 0 2 0 0 0 1 2 0 0 0 0 0 1 0 1 2 0 0 0 1 2 2 1 0 2 2 1 0 0 0 1 2 2 2 1 1
 0 1 0 0 2 0 0 0 0 0 2 2 0 2 1 0 2 2 0 0 2 2 2 1 1 0 0 0 2 2 1 0 0 1 2 1 0
 1 0 2 0 0 1 0 0 0 1 2 1 0 0 1 0 0 1 0 1 2]
